In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
df_train = pd.read_csv('/kaggle/input/dont-overfit-ii/train.csv')
df_test = pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv')
df_train.head()
df_test.head()

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
#check columns type 
for i in df_train.columns: 
    #print(type(i),end=' ') #change them from str to float to give them to the model
    df_train[i]= df_train[i].astype(float)
    
df_train.dtypes

### so we have a train data with only 250 entries while test data is 20,000 entries 
### I will try to oversampling train data but first check classes of it's target

In [ ]:
df_train_copy = df_train[['id','target']]
df_train_copy.groupby('target').count()

# Model Fitting before oversampling

In [ ]:
y= df_train['target']
labels= df_train.columns.drop(['id','target'])
X=df_train[labels]
X.head()
y.head()

# Split Data

In [ ]:
from sklearn.model_selection import train_test_split
  
# split into 70:30 ration
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
  
# logistic regression object
lr = LogisticRegression()
  
# train the model on train set
lr.fit(X_train, y_train)
  
predictions = lr.predict(X_test)
  
# print classification report
print(classification_report(y_test, predictions))
# print('auc_train:', roc_auc_score(y_train,predictions))
print('auc test before oversampling: {}'.format(roc_auc_score(y_test, predictions)))

# we can say it is imbaalanced a little bit dataset
### Oversampling — Duplicating samples from the minority class 
#### (I will try RandomOverSample In spite of it is a naive technique because when performed it assumes nothing of the data )

prepare X,y from train dataset


## We can not choose to do undersampling since entries are only 250 

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [ ]:
ros = RandomOverSampler()
X_ros , y_ros = ros.fit_resample(X,y)
print('Before resample : {}'.format(Counter(df_train['target']))) #another check
print('After resample : {}'.format(Counter(y_ros)))

# Split train data Using Random Sampling

In [ ]:
# split into 70:30 ration
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros, test_size = 0.3, random_state = 0)


In [ ]:
# logistic regression object
lr = LogisticRegression()
  
# train the model on train set
lr.fit(X_train, y_train)
  
predictions = lr.predict(X_test)
  
# print classification report
print(classification_report(y_test, predictions))
# print('auc_train:', roc_auc_score(y_train,predictions))
print('auc_test Using SmoteOverSampling using Random Sample:{}'.format(roc_auc_score(y_test, predictions)))

## SMOTE synthesises new minority instances between existing minority instances. It generates the virtual training records by linear interpolation for the minority class. 

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X, y)
print('Before resample : {}'.format(Counter(df_train['target']))) #another check
print('After resample : {}'.format(Counter(y_train_res)))

# Split train data Using SMOTE Sampling

In [ ]:
# split into 70:30 ration
X_train, X_test, y_train, y_test = train_test_split(X_train_res, y_train_res, test_size = 0.3, random_state = 0)


In [ ]:
# logistic regression object
lr = LogisticRegression()
  
# train the model on train set
lr.fit(X_train, y_train)
  
predictions = lr.predict(X_test)
  
# print classification report
print(classification_report(y_test, predictions))
# print('auc_train:', roc_auc_score(y_train,predictions))
print('auc_test Using SmoteOverSampling:{}'.format(roc_auc_score(y_test, predictions)))

# SmoteOverSampling is better than randomOverSampling in our case 

# Try Test data

In [ ]:
df_test.head()

In [ ]:
ide = df_test['id']
labels= df_test.columns.drop('id')
X_test_data=df_test[labels]
X_test_data.head()

In [ ]:
predictions = lr.predict(X_test_data)
output=pd.DataFrame({'id':ide,
                    'target':predictions})
output.head()

In [ ]:
output.to_csv('submission.csv', index=False)